In [ ]:
!pip install jupyter-dash
!pip install dash

     |████████████████████████████████| 7.3 MB 6.3 MB/s 
     |████████████████████████████████| 25.3 MB 59.8 MB/s 
     |████████████████████████████████| 357 kB 44.2 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=a84f177d7384c02e0e21ff448dfeadd275acb5cd42c6724511fd463f473e030f
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=deddd6f17dbfaa48c9e26ba51f474ee91ded1cca1b44e7ce7299c971e8af6166
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=eeed2acfff82401a247cc8a5199589ee2aebd113fee05ba34a2dcf97a805322d
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663ae

In [ ]:
from jupyter_dash import JupyterDash

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
from jupyter_dash import jupyter_dash

import plotly.express as px
import plotly.io as pio

import plotly.graph_objects as go

from dash.dependencies import Input,Output,State

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
df = pd.read_csv('/content/RFM_data_normalized.csv')
print(df['Recency'].max())
print(df['Recency'].min())
print(df['Frequency'].max())
print(df['Frequency'].min())
print(df['Monetary_Value'].max())
print(df['Monetary_Value'].min())

373
0
7847
1
280206.02
0.0


In [ ]:
df

,Unnamed: 0,CustomerID,Recency,Frequency,Monetary_Value,recency_normalized,frequency_normalized,monetary_value_normalized
0,0,12346.0,325,1,77183.60,49,1,50
1,1,12347.0,2,182,4310.00,2,45,47
2,2,12348.0,75,31,1797.24,31,21,39
3,3,12349.0,18,73,1757.55,13,34,39
4,4,12350.0,310,17,334.40,48,13,15
...,...,...,...,...,...,...,...,...
4334,4334,18280.0,277,10,180.60,46,7,7
4335,4335,18281.0,180,7,80.82,41,5,2
4336,4336,18282.0,7,12,178.05,6,9,7
4337,4337,18283.0,3,756,2094.88,3,50,41


In [ ]:
fig = px.scatter_3d(
      
      data_frame = df,
      x = df['Recency'],
      y = df['Frequency'],
      z = df['Monetary_Value'],
      color = df['recency_normalized'],
      
      
      # log_x = True,
      template = "plotly_dark",
      title="3D Scatterplot for RFM data",
      hover_name=df['CustomerID'],
      hover_data={ 'Monetary_Value':':.1f' },
      height= 700,

  


  
)






# Use for animation rotation at the end
x_eye = -1.25
y_eye = 2
z_eye = 0.5


#animation rotation code

fig.update_layout(scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
                  updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                  method='animate',
                                                  args=[None, dict(frame=dict(duration=100, redraw=True),
                                                                   transition=dict(duration=0),
                                                                   fromcurrent=True,
                                                                   mode='immediate'
                                                                   )]
                                                  )
                                             ]
                                    )
                               ]
                  )

def rotate_z(x, y, z, theta):
    w = x + 1j * y
    return np.real(np.exp(1j * theta) * w), np.imag(np.exp(1j * theta) * w), z


frames = []

for t in np.arange(0, 6.26, 0.1):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))

fig.frames = frames
#-------------------------------------------------------------------------------
pio.show(fig)


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([


#------------------------scatter code start------------------------------------

dcc.Graph(figure=fig),
#------------------------scatter code end-------------------------------------



#------------------------dropdown code start------------------------------------
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in df.columns],
                value='Recency',
                clearable=False,
            ),dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
            ],style={'width': '33.33%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in df.columns],
                value='Monetary_Value',
                clearable=False,
            ),dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
            ], style={'width': '33.33%','display': 'inline-block'}),
        
         html.Div([
            dcc.Dropdown(
                id='color-seperating-column',
                options=[{'label': i, 'value': i} for i in df.columns],
                value='recency_normalized',
                clearable=False,
            )],style={'width': '33.33%','float': 'right','display': 'inline-block'}),



    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
#------------------------dropdown code end-------------------------------------

#------------------------scatter chart code start-------------------------------------
 html.Div([
        dcc.Graph(
            id='scatter-chart',
            figure ={})
    ], style={'width': '50%', 'display': 'inline-block', 'padding': '0 20'}),

#------------------------scatter chart code end-------------------------------------


#------------------------bar chart code start-------------------------------------
 html.Div([
        dcc.Graph(
            id='bar-chart',
            figure ={})
    ], style={'width': '50%', 'display': 'inline-block', 'padding': '0 20'}),

#------------------------bar chart code end-------------------------------------


])


@app.callback(Output('scatter-chart', 'figure'),
              [Input('xaxis-column', 'value'),
              Input('yaxis-column', 'value'),
              Input('xaxis-type', 'value'),
              Input('yaxis-type', 'value'),
              Input('color-seperating-column', 'value')]
              )

def update_scatter_chart(xaxis_value,yaxis_value,xaxis_type,yaxis_type,color_value):
  if len(xaxis_value) == 0 or len(yaxis_value) == 0:
    return dash.no_update

  else: 
    fig = px.scatter(df,
                    x=df[xaxis_value],
                    y=df[yaxis_value],
                    hover_name=df['CustomerID'],
                    hover_data={ 'CustomerID':True,'CustomerID':':.1f','Monetary_Value':':.1f','Recency':True,'Frequency':True},
                    color = df[color_value],
                    template = 'ggplot2'
                 
                    
    )

    fig.update_xaxes(title=xaxis_value, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_value, type='linear' if yaxis_type == 'Linear' else 'log')
    
    return fig




@app.callback(Output('bar-chart', 'figure'),
              [Input('xaxis-column', 'value'),
              Input('yaxis-column', 'value'),
              Input('xaxis-type', 'value'),
              Input('yaxis-type', 'value'),
              Input('color-seperating-column', 'value')]
              )




def update_bar_chart(xaxis_value,yaxis_value,xaxis_type,yaxis_type,color_value):
  if len(xaxis_value) == 0 or len(yaxis_value) == 0:
    return dash.no_update

  else: 
    figg = px.bar(df,
                    x=df[xaxis_value],
                    y=df[yaxis_value],
                    hover_name=df['CustomerID'],
                    hover_data={ 'CustomerID':True,'CustomerID':':.1f','Monetary_Value':':.1f','Recency':True,'Frequency':True},
                    color = df[color_value],
                    template = 'ggplot2'
                 
                    
    )

    figg.update_xaxes(title=xaxis_value, type='linear' if xaxis_type == 'Linear' else 'log')

    figg.update_yaxes(title=yaxis_value, type='linear' if yaxis_type == 'Linear' else 'log')
    
    return figg
 
    











In [ ]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
app.run_server(mode="inline")

<IPython.core.display.Javascript object>